In [1]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import csv
import json
import os
import re
from datetime import datetime

import numpy as np
import pandas as pd

In [3]:
from fpl import get_player_data, get_fixtures
from optimize import optimize

(CVXPY) Aug 28 05:59:19 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Aug 28 05:59:19 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Aug 28 05:59:19 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/somiandras/Code/da

In [4]:
df = get_player_data(my_team_path="../sample_data/team_data.json")
df.head()

,name,position,team,value,expected_points,in_squad,is_available
id,,,,,,,
1,Cédric,DEF,ARS,43.0,0.000000,False,True
3,Xhaka,MID,ARS,50.0,19.038462,False,True
4,Elneny,MID,ARS,44.0,1.730769,False,True
5,Holding,DEF,ARS,43.0,0.865385,False,True
6,Partey,MID,ARS,50.0,6.923077,False,False


In [8]:
optimize(df, 20, 15)

/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 22 times so far.

  warnings.warn(msg, UserWarning)
/Users/somiandras/Code/data-projects/fpl_2022/env/lib/python3.10/site-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hi

SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer linear programming
                    solver `GLPK`. If your problem is nonlinear then you can install SCIP
                    (pip install pyscipopt).

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

In [68]:
df["position"].sort_values().unique()

array(['DEF', 'FWD', 'GKP', 'MID'], dtype=object)

In [47]:
fixtures = pd.DataFrame.from_records([fixture.dict() for fixture in get_fixtures()])
(
    pd.concat(
        [
            fixtures[["id", "event", "team_h", "team_h_difficulty"]].rename(
                columns={"team_h": "team", "team_h_difficulty": "difficulty"}
            ),
            fixtures[["id", "event", "team_a", "team_a_difficulty"]].rename(
                columns={"team_a": "team", "team_a_difficulty": "difficulty"}
            ),
        ]
    )
    .query("event < 10")
    .assign(phase=lambda x: (x["event"] < 5).map({True: "pre", False: "post"}))
    .groupby(["team", "phase"])["difficulty"]
    .mean()
    .unstack(1)
    .pipe(lambda x: x["pre"] / x["post"])
)


team
1     0.865385
2     0.661765
3     1.704545
4     1.250000
5     1.071429
6     0.892857
7     1.339286
8     0.781250
9     1.083333
10    1.153846
11    1.041667
12    0.961538
13    1.041667
14    0.859375
15    0.982143
16    0.916667
17    1.354167
18    0.735294
19    0.982143
20    0.808824
dtype: float64